<a href="https://colab.research.google.com/github/imkaywu/notebooks/blob/main/movie_text_match.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os

os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/MyDrive/Colab Notebooks/kaggle"

## Preprocess movie dataset

In [3]:
!kaggle datasets download -d cryptexcode/mpst-movie-plot-synopses-with-tags

 90% 26.0M/28.8M [00:02<00:00, 17.0MB/s]
100% 28.8M/28.8M [00:02<00:00, 12.2MB/s]


In [4]:
!mkdir -p movie_dataset && unzip \*.zip -d movie_dataset && rm *.zip

Archive:  mpst-movie-plot-synopses-with-tags.zip
  inflating: movie_dataset/mpst_full_data.csv  
  inflating: movie_dataset/partition.json  


In [5]:
import pandas as pd

df = pd.read_csv('movie_dataset/mpst_full_data.csv')

In [6]:
df = df.sort_values('title').reset_index(drop=True)
df['lev'] = None
df

,imdb_id,title,plot_synopsis,tags,split,synopsis_source,lev
0,tt0068152,$,"Set in Hamburg, West Germany, several criminal...",murder,test,imdb,None
1,tt0190938,$windle,A 6th grader named Griffin Bing decides to gat...,flashback,train,wikipedia,None
2,tt2614684,'71,"Gary Hook, a new recruit to the British Army, ...","suspenseful, neo noir, murder, violence",train,wikipedia,None
3,tt0085127,'A' gai wak,Sergeant Dragon Ma (Jackie Chan) is part of th...,"cult, violence",train,wikipedia,None
4,tt0080310,'Breaker' Morant,"In Pretoria, South Africa, in 1902, Major Char...","murder, anti war, violence, flashback, tragedy...",train,wikipedia,None
...,...,...,...,...,...,...,...
14823,tt3543634,Ängelby,Vera Fors lives an ordinary life with her husb...,"paranormal, murder",train,wikipedia,None
14824,tt0065261,Ådalen 31,"In 1931, the working-class family Andersson of...",romantic,val,wikipedia,None
14825,tt3425936,Él,The film opens during a foot washing ceremony ...,autobiographical,train,wikipedia,None
14826,tt0235198,Ôdishon,"Shigeharu Aoyama (Ryo Ishibashi), a middle-age...","cruelty, murder, cult, violence, flashback, ps...",train,imdb,None


In [7]:
!pip install levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 78.6 MB/s eta 0:00:00


In [8]:
from Levenshtein import distance

for a in range(len(df)-1):
    if distance(df.iloc[a].title, df.iloc[a+1].title) <= 3:
        print(a, df.iloc[a].title, df.iloc[a+1].title)
        df.at[a, 'lev'] = distance(df.iloc[a].title, df.iloc[a+1].title)
df
#we filter similar movies
df = df[df['lev'].isnull()].reset_index(drop=True)
df

23 1 10
37 11-11-11 11/11/11
40 12 Angry Men 12 Angry Men
43 12 Rounds 12 Rounds
52 13 Hours 13 Hrs
75 20,000 Leagues Under the Sea 20,000 Leagues Under the Sea
82 2012 2046
83 2046 21
87 21 Jump Street 22 Jump Street
89 23 24
90 24 24
127 3:10 to Yuma 3:10 to Yuma
169 8MM 8MM 2
171 8½ 9
184 A Boy and His Dog A Boy and His Dog
189 A Bug's Life A Bug's Life
201 A Christmas Carol A Christmas Carol
225 A Farewell to Arms A Farewell to Arms
253 A Haunted House A Haunted House 2
276 A Matter of Life and Death A Matter of Loaf and Death
278 A Matter of Time A Matter of Time
280 A Midsummer Night's Dream A Midsummer Night's Dream
281 A Midsummer Night's Dream A Midsummer Night's Dream
282 A Midsummer Night's Dream A Midsummer Night's Dream
283 A Midsummer Night's Dream A Midsummer Night's Dream
297 A Nightmare on Elm Street A Nightmare on Elm Street
298 A Nightmare on Elm Street A Nightmare on Elm Street
320 A Series of Unfortunate Events A Series of Unfortunate Events
332 A Star Is Bored A S

,imdb_id,title,plot_synopsis,tags,split,synopsis_source,lev
0,tt0068152,$,"Set in Hamburg, West Germany, several criminal...",murder,test,imdb,None
1,tt0190938,$windle,A 6th grader named Griffin Bing decides to gat...,flashback,train,wikipedia,None
2,tt2614684,'71,"Gary Hook, a new recruit to the British Army, ...","suspenseful, neo noir, murder, violence",train,wikipedia,None
3,tt0085127,'A' gai wak,Sergeant Dragon Ma (Jackie Chan) is part of th...,"cult, violence",train,wikipedia,None
4,tt0080310,'Breaker' Morant,"In Pretoria, South Africa, in 1902, Major Char...","murder, anti war, violence, flashback, tragedy...",train,wikipedia,None
...,...,...,...,...,...,...,...
12953,tt3543634,Ängelby,Vera Fors lives an ordinary life with her husb...,"paranormal, murder",train,wikipedia,None
12954,tt0065261,Ådalen 31,"In 1931, the working-class family Andersson of...",romantic,val,wikipedia,None
12955,tt3425936,Él,The film opens during a foot washing ceremony ...,autobiographical,train,wikipedia,None
12956,tt0235198,Ôdishon,"Shigeharu Aoyama (Ryo Ishibashi), a middle-age...","cruelty, murder, cult, violence, flashback, ps...",train,imdb,None


In [9]:
#find Avengers duplicates
for a in range(len(df)):
    if df.iloc[a]['title'].find('Avengers') != -1:
        pass
        #print(a)
#drop extra
df = df.drop([9572]).reset_index(drop=True) #i can do 1, 2, 3... to drop multiple
df
df.to_csv('movie_dataset/mpst_no_duplicates.csv')

## Encode data

In [11]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.0 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=6e612684ebf3300becb0627480495edbad0ffa222e3bb90e4907e981552aa978
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [12]:
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util
import numpy as np

tqdm.pandas()

model = SentenceTransformer('all-MiniLM-L6-v2')

df_ = df.copy()
df_['plot_synopsis'] = df_['plot_synopsis'].progress_apply(lambda x : model.encode(x))
df_index = df_.pop('title')
df_ = df_[['plot_synopsis']]
df_ = pd.DataFrame(np.column_stack(list(zip(*df_.values))))
df_.index = df_index
df_

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

100%|██████████| 12957/12957 [02:40<00:00, 80.57it/s]


,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
title,,,,,,,,,,,,,,,,,,,,,
$,-0.024897,0.055245,-0.122327,-0.029603,-0.007433,0.036262,0.129328,0.020940,0.048176,-0.037479,...,0.032808,-0.081902,-0.002921,0.033075,0.036488,0.016339,0.081623,-0.065515,0.016297,-0.047112
$windle,-0.032737,0.183432,-0.103036,-0.038543,-0.011393,0.050009,0.066918,0.036443,0.018850,0.002050,...,0.012507,-0.092158,-0.028933,-0.058818,0.019628,-0.001966,0.089710,0.011076,-0.016599,0.075246
'71,-0.080760,0.002873,-0.099334,-0.058721,0.020339,0.047407,0.044057,-0.010748,-0.049804,0.035990,...,-0.013720,-0.032820,0.007075,0.006609,-0.014338,0.049984,-0.021645,-0.003516,-0.011251,-0.024905
'A' gai wak,-0.114455,0.049236,0.034111,0.006510,-0.012836,0.074500,0.064006,-0.022156,-0.054337,0.011865,...,-0.042240,0.089022,0.016201,0.042965,-0.108093,0.070876,0.028978,-0.021293,-0.020478,0.043706
'Breaker' Morant,-0.114930,0.037941,-0.024508,-0.044298,0.029797,0.022180,0.019458,0.065946,0.001178,0.091679,...,0.025499,0.035366,0.037543,-0.014330,-0.054029,0.015875,-0.022152,-0.068856,-0.022993,-0.009707
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ängelby,-0.078154,0.015196,-0.039107,0.038870,0.102235,0.062958,0.015751,0.002622,0.064475,-0.005267,...,0.019878,0.011939,-0.001249,0.057983,-0.032008,0.108395,-0.001856,0.017713,-0.019270,-0.021597
Ådalen 31,-0.070786,0.102453,-0.025706,0.000993,0.014982,0.061297,0.107749,-0.013328,-0.007382,0.016728,...,0.006848,-0.029199,0.007583,-0.063745,-0.061957,0.053530,0.007582,0.002616,-0.015631,0.048152
Él,-0.032808,0.009943,0.047571,0.073746,0.043413,0.080470,-0.037416,0.024547,0.134191,-0.069619,...,0.038110,-0.026639,-0.023677,0.021118,0.026846,0.043454,-0.043832,0.004737,-0.037642,-0.059959


In [13]:
df_.to_csv('movie_dataset/mpst_encoded_no_duplicates.csv')

## Perform NN search

In [14]:
df_movies_encoded = pd.read_csv('movie_dataset/mpst_encoded_no_duplicates.csv')
df_movies_encoded.index = df_movies_encoded.pop('title')
df_movies_encoded

,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
title,,,,,,,,,,,,,,,,,,,,,
$,-0.024897,0.055245,-0.122327,-0.029603,-0.007433,0.036262,0.129328,0.020940,0.048176,-0.037479,...,0.032808,-0.081902,-0.002921,0.033075,0.036488,0.016339,0.081623,-0.065515,0.016297,-0.047112
$windle,-0.032737,0.183432,-0.103036,-0.038543,-0.011393,0.050009,0.066918,0.036443,0.018850,0.002050,...,0.012507,-0.092158,-0.028933,-0.058818,0.019628,-0.001966,0.089710,0.011076,-0.016599,0.075246
'71,-0.080760,0.002873,-0.099334,-0.058721,0.020339,0.047407,0.044057,-0.010748,-0.049804,0.035990,...,-0.013720,-0.032820,0.007075,0.006609,-0.014338,0.049984,-0.021645,-0.003516,-0.011251,-0.024905
'A' gai wak,-0.114455,0.049236,0.034111,0.006510,-0.012836,0.074500,0.064006,-0.022156,-0.054337,0.011865,...,-0.042240,0.089022,0.016201,0.042965,-0.108093,0.070876,0.028978,-0.021293,-0.020478,0.043706
'Breaker' Morant,-0.114930,0.037941,-0.024508,-0.044298,0.029797,0.022180,0.019458,0.065946,0.001178,0.091679,...,0.025499,0.035366,0.037543,-0.014330,-0.054029,0.015875,-0.022152,-0.068856,-0.022993,-0.009707
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ängelby,-0.078154,0.015196,-0.039107,0.038870,0.102235,0.062958,0.015751,0.002622,0.064475,-0.005267,...,0.019878,0.011939,-0.001249,0.057983,-0.032008,0.108395,-0.001856,0.017713,-0.019270,-0.021597
Ådalen 31,-0.070786,0.102453,-0.025706,0.000993,0.014982,0.061297,0.107749,-0.013328,-0.007382,0.016728,...,0.006848,-0.029199,0.007583,-0.063745,-0.061957,0.053530,0.007582,0.002616,-0.015631,0.048152
Él,-0.032808,0.009943,0.047571,0.073746,0.043413,0.080470,-0.037416,0.024547,0.134191,-0.069619,...,0.038110,-0.026639,-0.023677,0.021118,0.026846,0.043454,-0.043832,0.004737,-0.037642,-0.059959


### Method 1

In [15]:
from Levenshtein import distance
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd

nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(df_movies_encoded)
#string-searching algorithm
def closest_title(title):
    m = pd.DataFrame(df_movies_encoded.index)
    m['lev'] = m['title'].apply(lambda x : distance(x, title))
    return m.sort_values('lev', ascending=True)['title'].iloc[0]

In [16]:
def find_similar_movies(df, nbrs, title):
    #if title not in df it will choose the best search
    title = closest_title(title)

    distances, indices = nbrs.kneighbors([df.loc[title]])
    #print(indices)
    #we print df data, no longer df_
    for index in indices[0][1:]:
        print('index', index)
        print(title, '->', df.iloc[index].name)

In [17]:
find_similar_movies(df_movies_encoded, nbrs, 'Prince of Egypt')

index 11440
The Prince of Egypt -> The Ten Commandments: The Musical


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


### Method 2

In [32]:
embeddings = df_movies_encoded.to_numpy(dtype='float32')
embeddings.dtype

dtype('float32')

In [28]:
def calc_pairwise_dist(features1, features2):
  N, D = features1.shape
  T, _ = features2.shape
  assert features1.dtype == features2.dtype
  dtype = features1.dtype
  return np.matmul(features1**2, np.ones((D, T), dtype=dtype)) + np.matmul(np.ones((N, D), dtype=dtype), features2.T**2) - 2 * np.matmul(features1, features2.T)

# a = np.array([[1, 1, 1], [2, 2, 2], [3, 3, 3], [4, 4, 4]], dtype=int)
# b = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 10, 11]], dtype=int)
# calc_pairwise_dist(a, b)

array([[  2,  29, 110, 245],
       [  5,  14,  77, 194],
       [ 14,   5,  50, 149],
       [ 29,   2,  29, 110]])

In [47]:
def find_similar_movies(query, top_k):
  query_embedding = model.encode(query)

  dist = calc_pairwise_dist(embeddings, query_embedding[None, :])
  dist = np.squeeze(dist, axis=1)
  indices = np.argsort(dist)
  return df.iloc[indices[:top_k]]

In [48]:
query = '''The Avengers follows the aftermath of several previous Marvel Cinematic Universe (MCU) films, where various superheroes have been introduced, each with their own unique abilities and backgrounds. The movie opens with the Tesseract, a powerful energy source, falling into the hands of the malevolent Loki (Tom Hiddleston), who plans to use it to conquer Earth and subjugate humanity. In response to the imminent threat, Nick Fury (Samuel L. Jackson), the director of S.H.I.E.L.D., assembles a team of extraordinary individuals to thwart Loki’s nefarious plans. This team, known as the Avengers, comprises Iron Man, Captain America, Thor, Hulk, Black Widow, and Hawkeye. However, their initial inability to work together as a cohesive unit threatens to jeopardize their mission. As the Avengers confront Loki and his army of Chitauri invaders in New York City, they must overcome their differences and egos to forge a formidable alliance. Together, they showcase their unique strengths and talents in a pulse-pounding battle to protect Earth and its inhabitants from ultimate destruction.'''

find_similar_movies(query, 3)

,imdb_id,title,plot_synopsis,tags,split,synopsis_source,lev
887,tt0203247,Avengers,"The Asgardian Loki encounters the Other, the l...",good versus evil,train,wikipedia,None
9572,tt1626038,The Avengers: Earth's Mightiest Heroes,=== Season One ===\nAs 75 of the world's most ...,violence,test,wikipedia,None
1764,tt3498820,Captain America: Civil War,"In 1991, Bucky Barnes (Sebastian Stan), brainw...","suspenseful, murder, violence, intrigue, flash...",train,imdb,None


In [49]:
query = '''chris evans #chrisevans #paulrudd #captainamerica #antman #theavengers #avengers #marvel #sebastianstan'''

find_similar_movies(query, 3)

,imdb_id,title,plot_synopsis,tags,split,synopsis_source,lev
9572,tt1626038,The Avengers: Earth's Mightiest Heroes,=== Season One ===\nAs 75 of the world's most ...,violence,test,wikipedia,None
1764,tt3498820,Captain America: Civil War,"In 1991, Bucky Barnes (Sebastian Stan), brainw...","suspenseful, murder, violence, intrigue, flash...",train,imdb,None
1767,tt1843866,Captain America: The Winter Soldier,"Two years after the events of 'The Avengers', ...","murder, violence, flashback, good versus evil,...",val,imdb,None


In [54]:
query = 'Ayo this thing racist #spiderman #avengers #marvel #marvelstudios #funny #foryou'

find_similar_movies(query, 3)

,imdb_id,title,plot_synopsis,tags,split,synopsis_source,lev
9572,tt1626038,The Avengers: Earth's Mightiest Heroes,=== Season One ===\nAs 75 of the world's most ...,violence,test,wikipedia,None
96,tt0181947,3 dev adam,"The story takes place in Istanbul, where a vio...",cult,train,wikipedia,None
8934,tt1629701,Spider-Man: Shattered Dimensions,"=== Setting ===\nAn artifact known as the ""Tab...","violence, sci-fi",train,wikipedia,None


In [52]:
df.iloc[96].plot_synopsis

'The story takes place in Istanbul, where a violent criminal organization led by Spider-Man ("Spider\'s Gang") surfaces in the city with counterfeit United States dollars. A small task-force consisting of Captain America, Santo and Captain America\'s girlfriend Julia arrives to help local police stop Spider-Man and his gang.\nCaptain America\'s girlfriend, who has infiltrated Spider-Man\'s hideout, is captured and taken to a house in a remote location. She manages to send an SOS signal to the Captain. Captain America saves Julia and chases after Spider-Man, who manages to escape.\nMeanwhile, Mexico\'s national superhero/wrestler, Santo, infiltrates the dojo that is used as a front for counterfeiting. After being captured, he manages to escape along with incriminating evidence.\nCaptain and Santo raid a very important hideout where most of the counterfeiting operation is taking place. They manage to shut down the hideout while Spider-Man kills a couple, steals a statue and runs away.\nS

In [57]:
query = 'It focuses on the transformation of his youngest son, Michael Corleone, from reluctant family outsider to ruthless mafia boss.'

find_similar_movies(query, 3)

,imdb_id,title,plot_synopsis,tags,split,synopsis_source,lev
10214,tt0099674,The Godfather: Part III,"The movie begins in 1979, with a brief flashba...","murder, dramatic, cult, flashback, good versus...",train,imdb,None
10213,tt1198207,The Godfather II,"In 1901, the family of nine-year-old Vito Ando...","violence, humor, murder",train,wikipedia,None
4227,tt0070165,Heavy Traffic,"The film starts out in live action, introducin...","pornographic, cult, violence, psychedelic, hum...",train,wikipedia,None


In [60]:
df.iloc[4227].plot_synopsis

'The film starts out in live action, introducing the protagonist Michael Corleone, a 22-year-old virgin (inaccurately referred to as 24 in the movie\'s trailer) playing pinball in New York City. The scene then transitions into animation. New York has a diseased, rotten, tough, and violent atmosphere. Michael\'s Italian father, Angelo "Angie" Corleone, is a struggling mafioso who frequently cheats on Michael\'s Jewish mother, Ida. The couple constantly bicker and try to kill each other. Michael ambles through a catalog of freaks, greasers, and dopers. Unemployed, he dabbles with cartoons, artistically feeding off the grubbiness of his environment. He regularly hangs out at a local bar where he gets free drinks from the female black bartender, Carole, in exchange for sketches of the somewhat annoying Shorty, Carole\'s violent, legless bouncer devotee. One of the regular customers at the bar named Snowflake, a nymphomaniac transvestite, gets beaten up by a tough drunk hard-hat who has onl

In [61]:
query = '''This is Quentin Tarantino at his finest! Engaging characters; non-stop action; and intricately woven individual stories that produce a mesmerizing whole.

The depth of the overall plot is astounding. One minute dark and moody, the next riddled with black comedy and unforgettable utterances that remain noteworthy
quotes even into present day. Violence and blood aplenty, rampant drug use, startlingly intense situations- all signature Tarantino. And yet, this film is interspersed with sparkling nuggets of stark morality and unexpected wisdom. John Travolta as Vincent Vega, Samuel L. Jackson as Jules, Uma Thurman as Mia Wallace, and Bruce Willis as Butch are exceptionally complex characters who must learn to survive in an often cold and brutal world that still has hidden gems of humanity at its most basic fineness. Pulp Fiction is a roller coaster ride that will leave you gasping for more!'''

find_similar_movies(query, 3)

,imdb_id,title,plot_synopsis,tags,split,synopsis_source,lev
7626,tt0119942,Primary Colors,"Well acted drama, adapted from a novel by Joe ...","satire, atmospheric",train,imdb,None
6505,tt0077942,Moment by Moment,This synopsis is too short and may not include...,romantic,train,imdb,None
8202,tt0083015,S.O.B.,The story is a satire of the film industry and...,satire,train,wikipedia,None


In [67]:
query = "I haven't seen The Godfather in years but as it was on BBC2 last night, 13 December, decided to watch it again.  So glad I did as it is a superb film.  Although made in 1971-72 it has not dated, as so many films of that decade do.  Performances are superb, notably Marlon Brando and Al Pacino.  Pacino's face is like a mask - no emotion is shown  and he can casually order an opponent's execution as sitting down to a meal.  I also love the stillness of the movie and how family life was vividly shown - the Italian love of good food and drink.  Colours are muted in browns and golds and Coppola made a masterpiece.  5 star rating."

find_similar_movies(query, 3)

,imdb_id,title,plot_synopsis,tags,split,synopsis_source,lev
5448,tt0077138,L'albero degli zoccoli,An absolutely engrossing film. A wonderful sli...,"romantic, horror, storytelling",train,imdb,None
4607,tt0238891,I cento passi,The film opens with Peppino as a small child s...,murder,train,wikipedia,None
10214,tt0099674,The Godfather: Part III,"The movie begins in 1979, with a brief flashba...","murder, dramatic, cult, flashback, good versus...",train,imdb,None


In [64]:
df.iloc[5448].plot_synopsis

'An absolutely engrossing film. A wonderful slice of life in rural Italy at the turn of the century. No great dramatics but a detailed examination of a year of peasant life. The characters, especially the young children are so endearing. A three hour film but never dull. No big name stars but each performance is so very real. A must see. The photography of northern Italy is so very atmospheric. The plot emphasizes the extreme poverty of the people but also how well they treat one another and the limitations of living in the homes and the land of a well to do but uncaring landlord. The only parts that are difficult to watch are the slaughtering of livestock that was so much a part of life at the time. The program not only shows the extreme poverty of the farmers abut also their remarkable ingenuity in dealing with the trials that they face. The challenges faced are those relatively unknown to us, we who are so well off here in America. You can see from this film why so many individuals 

In [68]:
df.iloc[4607].plot_synopsis

"The film opens with Peppino as a small child singing the popular song “Nel blu, dipinto di blu” with his brother in the back seat of a car on the way to a family gathering. The family is one of good standing in the social community and they are celebrating the fact that they have such a good life. In this scene the relationship between Peppino and his uncle Cesare is established. His uncle is a Don or Mafia boss in the small town of Cinisi where the story is set. In a scene soon after the happy family gathering we see don Cesare blown up by a car bomb which was planted by a rival Mafia boss. This ends Peppino’s time of innocence. Even as a small child he is thrust into the realities of life in the Mafia.\nAfter his uncle’s funeral he goes to a local painter, Stefano Venuti, who is also a very outspoken member of the communist party in Sicily, to paint a picture of Cesare. Stefano refuses to paint it but does not really give him a reason. He did not get along with Cesare when he was al